# Razonamiento y Planificacion Automatica
## Actividad 2
 * Hever Gonzalez Garcia
 * Nathalia Orozco Morales
 * Ivan Jesus Zepeda Gonzalez

__Descripcion__

Utilizar la estrategia de búsqueda heurística A* con el fin de resolver el problema del puzzle-8. Utiliza como heurística el número de fichas mal colocadas respecto al estado objetivo. Considera que el coste de cada movimiento es 1.

Se deberán crear tantas clases o estructuras de datos como sean necesarias para representar el espacio de estados y los nodos de exploración del árbol.

El código deberá ejecutarse una vez y alcanzar el resultado, indicando la secuencia de acciones a realizar para alcanzar el objetivo utilizando una notación sencilla. 
Por ejemplo: «mover 7 hacia la derecha» o «mover 5 hacia la izquierda».



In [1]:
import numpy as np
import random
from  queue import PriorityQueue


### Se definen los estados del tablero

Entendamos Tablero como estado. Pues se usa el estado del tablero. Cualquiera de los dos terminos es para referirse a lo mismo.

Se empieza por asignar los valores permitidos (0-9) en las celdas del tablero.

Se recibe el valor del tablero objetivo/goal y el del inicio.

Si no se recibe un tablero inicial, se genera aleatoriamente este.

In [2]:
#Se puede hacer un puzzle 8 o un puzzle 15, [o puzzle(numero*4)-1]
#Aumentando de 3 a 4, la dimension del tablero
DIMENSION_TABLERO = 3

In [3]:
INICIO=np.array([[2, 8, 3],[1, 6, 4],[7, 0, 5]])
FINAL=np.array([[1, 2, 3],[8, 0, 4],[7, 6, 5]])
INICIO2=np.array([[1, 0, 2],[6, 3, 4],[7, 5, 8]])
FINAL2=np.array([[1, 2, 3],[4, 5, 6],[7, 8, 0]])

In [4]:
#Se asigna valores a las celdas, utilizando el rango disponible 0-9
valor_celdas = list(random.sample(range(DIMENSION_TABLERO*DIMENSION_TABLERO),DIMENSION_TABLERO*DIMENSION_TABLERO))  
tablero_final = FINAL
tablero_inicial = INICIO

print("Inicio\n",tablero_inicial)
print("Objetivo\n",tablero_final)

Inicio
 [[2 8 3]
 [1 6 4]
 [7 0 5]]
Objetivo
 [[1 2 3]
 [8 0 4]
 [7 6 5]]


### Calcular Distancia Manhattan

La distancia Manhattan se obtiene mediante:
La suma del valor absoluto de la resta de objetivoX menos EstadoActualX, con el valor absoluto de la resta de objetivoY menos EstadoActualY.

En otras palabras, esta distancia _"si tomamos dos puntos p y q en una cuadrícula con coordenadas p=(p1,p2) y q=(q1,q2), la distancia Manhattan entre dichos puntos es la suma de los valores absolutos de las diferencias entre las coordenadas. Es decir:
d(p,q)=|q1-p1|+|q2-p2|"_     [1]

El metodo calcular_distancia_manhattan, recibe un nodo del cual obtiene valores necesarios.

Primero usando shape, se obtiene el tamano del puzzle (3x3)
se hace un for loop con estos valores como limites del rango
si el valor de la celda no es el campo vacio, se revisa el valor objetivo de esta celda para calcularse en la formula de la distancia manhattan. Usando valores del estado actual y el estado objetivo.

In [5]:
def calcular_distancia_manhattan(nodo):
    sum_manhattan = 0
    (dimx, dimy) = nodo.matrix.shape
    for row in range(dimx):
        for col in range(dimy):
            if nodo.matrix[col][row] != 0:
                x, y = posiciones_objetivo[nodo.matrix[col][row]]
                sum_manhattan += abs(x - row) + abs(y - col)
    return sum_manhattan

### Creacion del Nodo
El nodo contiene datos del estado actual, el nodo padre, nivel/profundidad y los movimientos permitidos.
Se definen varios metodos:
* el constructor __ __init__ __ : Construye un nodo con los valores recibidos, en caso que no reciba genera estos del metodo indice_de_celda_vacia
* __calcular_costo__: regresa el valor de costo de funcion
* __indice_de_celda_vacia__: return coordenadas. llama indice_de_celda con la posicion 0
* __indice_de_celda__: return coordenadas. mediante un forloop hace una busqueda dentro del tablero hasta encontrar el valor solicitado
* __se_puede_mover__: return boolean. Revisa los posibles movimientos de la celda en posicion a las coordenadas recibidas. Verificando que estas esten dentro de los limites del tablero
* se hace override del operador __ __lt__ __ "less than". Aqui regresa un booleano si al objeto a comparar tiene mayor coste heuristico. __Se puede filtrar desde aqui las acciones, haciendo un greedy__

Sus propiedades son:
matrix, parent, nivel, x, y, distancia_heuristica, costo_de_funcion,accion


In [6]:
class Nodo(object):

    def __init__(self, matrix, x=None, y=None, nuevox=None, nuevoy=None,nivel=0, parent=None ):
        self.matrix = matrix.copy()
        self.parent = parent
        self.nivel = nivel        # el nivel actual del tablero
        self.celda=0
        # intercambio el valor de la nueva posicion
        if x == None or y==None or nuevox ==None or nuevoy == None:
            (nuevox, nuevoy)= (x, y) = self.indice_de_celda_vacia()      
        #intercambiar valor de celdas en la matriz con indices nuevox,nuevoy por x,y
        valor = self.matrix[nuevox,nuevoy]
        self.matrix[nuevox,nuevoy] = self.matrix[x,y]
        self.matrix[x,y] = valor
        self.celda=valor
        # la nueva posición que ocupa la solución
        self.x = nuevox
        self.y = nuevoy     
        self.distancia_heuristica = calcular_distancia_manhattan(self) # can be replace with the hamming distance
        self.costo_de_funcion = self.distancia_heuristica + self.nivel        
        #Registrar el movimento de la celda(1,0),(0,1),(-1,0),(0,-1): arriba,izquierda,abajo,derecha
        self.accion=""
        
    def calcular_costo(self):        
        return self.costo_de_funcion
    
    def indice_de_celda_vacia (self):
        return self.indice_de_celda(0)

    def indice_de_celda (self, celda):
        for r in range(3):
            for c in range(3):
                if self.matrix[r][c] == celda:
                    return r, c
    
    def direccion(self,x,y):
        #El resultado es inverso, pues se usara para la casilla con numero, no para la celda vacia (con el 0)
        if (x==1 and y==0):
            self.accion="arriba" 
        elif (x==-1 and y==0):
            self.accion="abajo"
        elif (x==0 and y==1):
            self.accion="izquierda"
        elif (x==0 and y==-1):
            self.accion="derecha"
        else:
            self.accion="quieto? x:"+str(x)+", y:",str(y)

    def es_seguro_moverse(self, x,y):
        return (x >= 0 and x < self.matrix.shape[0] and y>= 0 and y < self.matrix.shape[1])
    
    def __lt__(self, nodo2):
        return (self.costo_de_funcion, self.distancia_heuristica) < (nodo2.costo_de_funcion, nodo2.distancia_heuristica)   
    

In [7]:
#en el bloques siguientes
posiciones_objetivo = {tablero_final[col][row]: (row, col) for col in range(3) for row in range(3)} #un diccionario con las posiciones de cada celda del tablero_final(objetivo)
#se hace un diccionario de coordenadas
#{1: (0, 0), 2: (1, 0), 3: (2, 0), 8: (0, 1), 0: (1, 1), 4: (2, 1), 7: (0, 2), 6: (1, 2), 5: (2, 2)}

## Imprimir rastro de estados
Esta funcion hace de forma recursiva una impresion de las celdas en cada nodo padre.

Se detiene cuando ya no tiene nodo padre, siendo el fin de la solucion tomada. 
Siendo asi en cada iteración del algoritmo, indica claramente el nodo que ha sido expandido!

In [8]:
def print_nodo(nodo):        
    if nodo != None:        
        print_nodo(nodo.parent)  
        if(nodo.accion!=""):
            print('\nPasos/Nivel de Profundidad: ',nodo.nivel)
            print("Se movio ",nodo.celda, " a ",nodo.accion)
        else:
            print("INICIO")
        print(nodo.matrix)
   

Aqui se define los movimientos posibles, mediante valores en los ejes, para columnas y filas.
Se hace un set de estados visitados para evitar repetir y hacer ciclos inecesarios.

Se hace un queue de nodos.
Al mismo tiempo se agrega el nodo inicial (nodo_actual) al queue con su costo


In [9]:
# movimientos posibles. su combinacion se hace en el bloque de abajo, en un for range(4), 
#intentando 4 sentidos de coordenadas (1,0),(0,1),(-1,0),(0,-1)
filas = (1,0,-1,0)
columnas = (0,1,0,-1)
visitados = set()

queue = PriorityQueue()

nodo_actual = Nodo(tablero_inicial,nivel=0)
print('Tablero Inicial')
print(nodo_actual.matrix)

nodo_final = Nodo(tablero_final,nivel=0)
print('Tablero Final')
print(nodo_final.matrix)

queue.put((nodo_actual.calcular_costo(),nodo_actual))

Tablero Inicial
[[2 8 3]
 [1 6 4]
 [7 0 5]]
Tablero Final
[[1 2 3]
 [8 0 4]
 [7 6 5]]


### Logica iterativa
habiendo llenado con 1 elemento en la queue, el siguiente while loop continuara hasta ser detenido.
siendo el break posible cuando el tablero sea igual a la solucion, comparando su 
distancia_heuruistica a 0. __[Entre mas corta, mas acercada esta al objetivo, ese es usualmente el camino a tomar por el algoritmo A* que es mezcla de Greedy y Dijkstra]__

El estado es transformado en un string, para almacenarse en visitados.

Aqui se va midiendo los posibles movimientos de la celda, mostrando las opciones con su peso heuristico. 
Si el estado se repite, se ignora y salta a la siguiente iteracion.
Si la heuristica es 0, se llego al objetivo y termina el loop
Si no existe en visitados, se agrega 
Por ultimo se miden los movimientos, Si es posible moverse a esa posicion, se crea un nuevo nodo que se agrega a la queue, junto con este el costo de ese estado/tablero.

Key, es la string-izacion de los tableros/estados, y guardar estos en visitados, para facil ubicacion en caso de que ya este repetido

In [10]:
#Queue es una tupla, conteniendo elementos inmutables, con un int como key, y un nodo como value.
#      nodo_actual.calcular_costo(),       nodo_actual
pasos_totales=0
while not queue.empty():
    next_item = queue.get()  
    
    key = ''.join([str(next_item[1].matrix[col,row]) for col in range(3) for row in range(3)])    
   #NOTA: IMPRIMIR AQUI, PUEDE MOSTRAR ESTADOS REPETIDOS
    print("#",pasos_totales,' Costo total Heuristico:',next_item[0],", Costo de Paso:", next_item[1].distancia_heuristica)
    print(next_item[1].matrix)    
    
    if key in visitados:
        continue        
    if  next_item[1].distancia_heuristica == 0:        
        break
    
    visitados.add(key)
    for i in range(4):
        if(next_item[1].es_seguro_moverse(next_item[1].x+columnas[i],next_item[1].y+filas[i])):     #Aqui llamo al __lt__?
            #        Nodo(self, matrix,             x=None,         y=None,         nuevox=None,                nuevoy=None,             nivel=0,              parent=None): 
            nodo_actual = Nodo(next_item[1].matrix, next_item[1].x, next_item[1].y, next_item[1].x+columnas[i], next_item[1].y+filas[i], next_item[1].nivel+1, next_item[1])            
            nodo_actual.direccion(columnas[i],filas[i])
            queue.put((nodo_actual.calcular_costo(),nodo_actual))
            pasos_totales+=1
            #Para imprimir TODOS los intentos, aun los repetidos, imprimir aqui los pasos


# 1  Costo total Heuristico: 5 , Costo de Paso: 5
[[2 8 3]
 [1 6 4]
 [7 0 5]]
# 2  Costo total Heuristico: 5 , Costo de Paso: 5
[[2 8 3]
 [1 6 4]
 [7 0 5]]
# 3  Costo total Heuristico: 5 , Costo de Paso: 5
[[2 8 3]
 [1 6 4]
 [7 0 5]]
# 4  Costo total Heuristico: 5 , Costo de Paso: 4
[[2 8 3]
 [1 0 4]
 [7 6 5]]
# 5  Costo total Heuristico: 5 , Costo de Paso: 4
[[2 8 3]
 [1 0 4]
 [7 6 5]]
# 6  Costo total Heuristico: 5 , Costo de Paso: 4
[[2 8 3]
 [1 0 4]
 [7 6 5]]
# 7  Costo total Heuristico: 5 , Costo de Paso: 4
[[2 8 3]
 [1 0 4]
 [7 6 5]]
# 8  Costo total Heuristico: 5 , Costo de Paso: 3
[[2 0 3]
 [1 8 4]
 [7 6 5]]
# 9  Costo total Heuristico: 5 , Costo de Paso: 3
[[2 0 3]
 [1 8 4]
 [7 6 5]]
# 10  Costo total Heuristico: 5 , Costo de Paso: 3
[[2 0 3]
 [1 8 4]
 [7 6 5]]
# 11  Costo total Heuristico: 5 , Costo de Paso: 2
[[0 2 3]
 [1 8 4]
 [7 6 5]]
# 12  Costo total Heuristico: 5 , Costo de Paso: 2
[[0 2 3]
 [1 8 4]
 [7 6 5]]
# 13  Costo total Heuristico: 5 , Costo de Paso: 1
[[1 2 3]
 

Algunos numeros (#2,#3...), no son visibles, pues cuenta los pasos, aun cuando este estado ya sea repetido o agregado a la queue

## Solucion

La visualizacion de la solucion se da llamando el metodo __print_nodo__ enviandole el ultimo item que se registro. Este ultimo item, contiene todos los nodos previos en el atributo parent. Y de forma recursiva, se ira imprimiendo cada nodo, hasta llegar a uno que no tenga parent.
Se muestra el nivel de profundidad o pasos, asi como la accion tomada para cada paso, y visualizacion de la matriz.

In [11]:
print("\n*************************************\n",
      "Solucion\n*************************************\n")      
print_nodo(next_item[1]) #Imprime el ultimo nodo de la queue, que fue con el que termino el loop. En otras palabras, la solucion.
print("\nOBJETIVO:")
print(next_item[1].matrix)
print("\n-------------------------------------\n",
      "Se completo el juego en ",next_item[1].nivel, " pasos.\n",
      "Movimientos intentados:",pasos_totales,
      "\n-------------------------------------\n")


*************************************
 Solucion
*************************************

INICIO
[[2 8 3]
 [1 6 4]
 [7 0 5]]

Pasos/Nivel de Profundidad:  1
Se movio  6  a  abajo
[[2 8 3]
 [1 0 4]
 [7 6 5]]

Pasos/Nivel de Profundidad:  2
Se movio  8  a  abajo
[[2 0 3]
 [1 8 4]
 [7 6 5]]

Pasos/Nivel de Profundidad:  3
Se movio  2  a  derecha
[[0 2 3]
 [1 8 4]
 [7 6 5]]

Pasos/Nivel de Profundidad:  4
Se movio  1  a  arriba
[[1 2 3]
 [0 8 4]
 [7 6 5]]

Pasos/Nivel de Profundidad:  5
Se movio  8  a  izquierda
[[1 2 3]
 [8 0 4]
 [7 6 5]]

OBJETIVO:
[[1 2 3]
 [8 0 4]
 [7 6 5]]

-------------------------------------
 Se completo el juego en  5  pasos.
 Movimientos intentados: 15 
-------------------------------------



## Comparativa
Se ha realizado este puzzle de diversas formas o approaches. 
Los principales fueron:
* 1. Crear Nodos y Queues manualmente, sin uso de ninguna libreria
* 2. Utilizar la libreria __PriorityQueues__ para gestionar el orden de los nodos
* 3. Utilizar la libreria __simpleai__ la cual presenta gran facilidad para resolver el problema

Decidimos optar por el approach 2, el cual nos podriamos enfocar en la logica del algoritmo de A*, la distancia Manhattan, y el proceso Heuristico, sin entrar en control de las queues

Aun asi, mostramos el mismo acercamiento con la libreria simpleai, para comparar nuestro resultado con esta.

# Conclusiones
La solucion de un puzzle-8 utilizando A*, es relativamente sencillo.
Se necesita crear una lista que almacene nodos, y los pasos para llegar de un estado inicial a un objetivo
El analisis se realiza cambiando un elemento (la celda vacia, o con 0) a posibles ubicaciones que
esten adyacentes de esta. Y se decide optar por la que presente el valor mas chico de la distancia Manhattan.


# Practica Grupal

* Todos los miembros se han integrado al trabajo del grupo
 - Si

* Todos los miembros participan activamente
 - Si

* Todos los miembros respetan otras ideas aportadas
 - Si

* Todos los miembros participan en la elaboración del informe
 - Si

* Me he preocupado por realizar un trabajo cooperativo con mis compañeros
 - Si

* Señala si consideras que algún aspecto del trabajo en grupo no ha sido adecuado


## Referencias y bibliografia
* https://elpais.com/elpais/2016/10/18/el_aleph/1476813443_840074.html
* Artificial Intelligence with Python, Oreilly
* - from simpleai.search import astar
* - from simpleai.search import SearchProblem